In [5]:
import sys
from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import (QSystemTrayIcon, QApplication, QLabel, QMainWindow, QPushButton, QWidget, 
                             QVBoxLayout, QTextEdit, QDialog, QComboBox, QAction, QLineEdit, QLabel, QInputDialog)
from PyQt5.QtGui import QIcon, QFont
from PyQt5.QtCore import Qt, QCoreApplication, QSettings, QPoint
import tkinter as tk
import PIL.Image
import numpy as np
import cv2
import io
import json
import inspect
import ctypes
from io import BytesIO
import glob
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output, Image, display
from google.cloud.vision_v1 import types
from google.cloud import vision
import mss
import pyperclip
import mss.tools
import qt_utils
from PyQt5.QtWidgets import QMessageBox

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/poorv/Downloads/ocr_scripts/keys/direct-outlook-270501-c05c2d97d1c6.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/poorv/Downloads/ocr/service_credentials.json"

In [6]:
def send_text_clipboard(text):
    pyperclip.copy(text)
    spam = pyperclip.paste()
    
def dtlh(path, lh):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    if type(path) == str:
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
            image = vision.Image(content=content)
    else:
        image = types.Image(content=cv2.imencode('.jpg', path)[1].tobytes())
        
    response = client.text_detection(
    image=image,
    image_context={"language_hints": lh},
    )
    texts = response.text_annotations
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

    try:
        return texts[0].description
    except IndexError:
        print('no text in image')
        
dtlh(r"/Users/poorv/Downloads/tamil/better pages/139.png", ["ta"])

'தோட்டம்\nஇது ஓர் அழகான தோட்டம். இந்தத் தோட்டத்தில் நிறைய\nபூச்செடிகளும் பல பழ மரங்களும் இருக்கின்றன. அதனால் நிறைய\nபறவைகளையும்\nவண்டுகளையும்\nஇங்கே\nபார்க்கலாம்.\nசின்னக்\nகுழந்தைகள் தங்கள் பெற்றோருடன் இங்கே வருவார்கள். இந்தத்\nதோட்டத்தில் மான்கள் கூட இருக்கின்றன. இந்தத் தோட்டத்தைப் பராமரிக்க\nநல்ல தோட்டக்காரர்கள் இருக்கிறார்கள். காலையிலிருந்து இரவுவரை\nஅவர்கள் இங்கே இருப்பார்கள். இங்கே வரும் குழந்தைகளுக்கு\nவாசனையான பூக்களையும் ருசியான பழங்களையும் கொடுப்பர்கள்.\nஅதனால் குழந்தைகள் இங்கே வர ஆசைப்படுவார்கள்.\nBharathy, UCB\n139\n'

In [7]:
width = 2560
height = 1600

class ocrWidget(QtWidgets.QWidget):
#     clicked = QtCore.pyqtSignal(object)
    
    def __init__(self, parent = None, lang_hint = []):
        super().__init__()
        self.setGeometry(0, 0, width, height)
        self.setWindowTitle(' ')
        self.lang_hint = lang_hint
        self.begin = QtCore.QPoint()
        self.end = QtCore.QPoint()
        self.parent = parent
        self.setWindowOpacity(0.3)
        QtWidgets.QApplication.setOverrideCursor(
            QtGui.QCursor(QtCore.Qt.CrossCursor)
        )
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)
        QtWidgets.QShortcut(
            QtGui.QKeySequence("Escape"), self, activated=self.on_Escape
        )

    def paintEvent(self, event):
        qp = QtGui.QPainter(self)
        qp.setPen(QtGui.QPen(QtGui.QColor('black'), 3))
        qp.setBrush(QtGui.QColor(128, 128, 255, 128))
        qp.drawRect(QtCore.QRect(self.begin, self.end))

    def mousePressEvent(self, event):
        self.begin = event.pos()
        self.end = self.begin
        self.update()
    
    @QtCore.pyqtSlot()
    def on_Escape(self):
        print("main esp exit")
#         self.close()
        self.closeAndReturn()
        

    def mouseMoveEvent(self, event):
        self.end = event.pos()
        self.update()
    
    def closeAndReturn(self):
        self.close()
        self.parent.setWindowOpacity(1.)
        return

    def mouseReleaseEvent(self, event):
        previous = self.parent
        self.close()

        x1 = min(self.begin.x(), self.end.x())
        y1 = min(self.begin.y(), self.end.y())
        x2 = max(self.begin.x(), self.end.x())
        y2 = max(self.begin.y(), self.end.y())
        
        if (y1 == y2 and x1 == x2):
            print("no region selected")
            self.closeAndReturn()
        
        with mss.mss() as sct:
            print("lang_hint:", self.lang_hint)
            # The screen part to capture            
            monitor = {"top": y1, "left":x1, "width": abs(x2 - x1) , "height": abs(y2 - y1)}
            output = "sct-{top}x{left}_{width}x{height}.png".format(**monitor)
            sct_img = sct.grab(monitor)
            try:
                img = PIL.Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX")
                img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
                im_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                ret,thresh = cv2.threshold(im_gray,127,255,cv2.THRESH_BINARY_INV)
#                 ocr = dtlh(thresh, ["hi"])
                ocr = dtlh(thresh, self.lang_hint)
                popup = False
                if ocr:
                    send_text_clipboard(ocr)
                    print(ocr)
                    if popup:
                        msg = QMessageBox()
                        msg.setText(ocr)
                        msg.setWindowTitle("Captured Text")
                        msg.setFont(QFont('Arial', 20)) 
                        x = msg.exec_()  # this will show our messagebox
                self.parent.setWindowOpacity(1.)
            except ValueError:
                print("invalid region selection. try again")
                self.closeAndReturn()

In [8]:
lang_hints_dict = {u'Afrikaans - Afrikaans': "af", 
u'Albanian - shqip': "sq", 
u'Arabic - العربية': "ar", 
u'Armenian - Հայ': "hy", 
u'Belorussian - беларускі': "be", 
u'Bulgarian - български': "bg", 
u'Catalan - Català': "ca", 
u'Chinese - 普通话': "zh", 
u'Croatian - Hrvatski': "hr", 
u'Czech - Čeština': "cs", 
u'Danish - Dansk': "da", 
u'Dutch - Nederlands': "nl", 
u'English  - English': "en", 
u'Estonian - Eesti keel': "et", 
u'Filipino - Filipino': "fil (or tl)", 
u'Finnish - Suomi': "fi", 
u'French  - Français': "fr", 
u'German - Deutsch': "de", 
u'Greek - Ελληνικά': "el", 
u'Gujarati - ગુજરાતી': "gu", 
u'Hebrew - עברית': "iw", 
u'Hindi - हिन्दी': "hi", 
u'Hungarian - Magyar': "hu", 
u'Icelandic - Íslenska': "is", 
u'Indonesian - Bahasa Indonesia': "id", 
u'Italian - Italiano': "it", 
u'Japanese - 日本語': "ja", 
u'Kannada - ಕನ್ನಡ': "kn", 
u'Khmer - ភាសាខ្មែរ': "km", 
u'Korean - 한국어': "ko", 
u'Lao - ລາວ': "lo", 
u'Latvian - Latviešu': "lv", 
u'Lithuanian - Lietuvių': "lt", 
u'Macedonian - Македонски': "mk", 
u'Malay - Bahasa Melayu': "ms", 
u'Malayalam - മലയാളം': "ml", 
u'Marathi - मराठी': "mr", 
u'Nepali - नेपाली': "ne", 
u'Norwegian - Norsk': "no", 
u'Persian - فارسی': "fa", 
u'Polish - Polski': "pl", 
u'Portuguese - Português': "pt", 
u'Punjabi - ਪੰਜਾਬੀ': "pa", 
u'Romanian - Română': "ro", 
u'Russian - Русский': "ru", 
u'Russian - Русский (старая орфография)': "ru-PETR1708", 
u'Serbian - Српски': "sr", 
u'Serbian - Српски (латиница)': "sr-Latn", 
u'Slovak - Slovenčina': "sk", 
u'Slovenian - Slovenščina': "sl", 
u'Spanish - Español': "es", 
u'Swedish - Svenska': "sv", 
u'Tamil - தமிழ்': "ta", 
u'Telugu - తెలుగు': "te", 
u'Thai - ไทย': "th", 
u'Turkish - Türkçe': "tr", 
u'Ukrainian - Українська': "uk", 
u'Vietnamese - Tiếng Việt': "vi", 
u'Yiddish - Yiddish': "yi"}

class Central(qt_utils.QMainWindow):
    def __init__(self, parent=None):
        super(Central, self).__init__(parent)
        self.pushButton = QPushButton("OCR")
        self.pushButton.clicked.connect(self.on_pushButton_clicked)
        self.dialogs = list()
        self.combo = QComboBox(self)
        self.populate_dict(self.combo)
        self.codes = set(lang_hints_dict.values())
        self.combo.activated.connect(self.handleActivated)
        self.lang_hint = []    
        self.multiButton = QPushButton("OCR Multiple languages")
        self.multiButton.clicked.connect(self.sd)
        self.selected = []
        layout = QVBoxLayout()
        layout.addWidget(self.combo)
        layout.addWidget(self.pushButton)
        layout.addWidget(self.multiButton)
        widget = QWidget()
        widget.setLayout(layout)
        widget.setFont(QFont('Arial', 11)) 
        self.setCentralWidget(widget)
        self._gui_restore()
    
    def on_pushButton_clicked(self, mult = False):
        if not mult:
            self.lang_hint = [lang_hints_dict.get(str(self.combo.currentText()))]
        print("button clicked")
        dialog = ocrWidget(self, self.lang_hint)
        self.dialogs.append(dialog)
        dialog.show()
        self.setWindowOpacity(0.)
    
    def handleActivated(self, index):
        self.lang_hint = [self.combo.itemData(index)]
        
    def populate_dict(self, combo : QComboBox):
        for key, value in lang_hints_dict.items():
            self.combo.addItem(key, value)
            
    def sd(self):
        info_string = """To enter multiple languages, enter the language code in a comma seperate list in order of importance. For example, 
    if you want to recognize a document with Arabic, English, and French in it, and (Arabic being most and French least important)
    enter ar,en,fr. For the full list of language codes see https://cloud.google.com/vision/docs/languages#supported-langs
    """
        text , ok = QInputDialog.getText(self,'InputDialog', info_string)
        if ok:
            lh = text.replace(" ", "").split(',')
            self.lang_hint.clear()
            for elem in lh:
                if elem not in self.codes:
                    print("the language code {0} cannot be found! check your codes!".format(elem))
                    return
                else:
                    self.lang_hint.append(elem)
            self.on_pushButton_clicked(True)

    
def main():
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    main = Central()
    main.show()
    sys.exit(app.exec_())
    
if __name__ == '__main__':
    main()

  OpenType support missing for "Arial", script 14
  OpenType support missing for "Arial", script 11
  OpenType support missing for "Arial", script 18
  OpenType support missing for "Arial", script 32
  OpenType support missing for "Arial", script 19
  OpenType support missing for "Arial", script 13
  OpenType support missing for "Arial", script 16
  OpenType support missing for "Arial", script 17


button clicked
lang_hint: ['ta']


/var/folders/4n/6vck65rd73d5bflh0f914j3m0000gn/T/ipykernel_73995/105359903.py:13: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  image = types.Image(content=cv2.imencode('.jpg', path)[1].tostring())


ஒரு காட்டில் விலங்குகள் எல்லாம் சேர்ந்து ஒரு யானையைத்
தலைவனாகத் தேர்ந்தெடுத்தன. ஆனால் ஒரு மானுக்கு அது
பிடிக்கவில்லை. அந்த மான் மற்ற விலங்குகளிடம், "யானையுடைய
உடம்பு ரொம்ப பெரிய உடம்பு; யானைக்குப் பெரிய தந்தம் இருக்கிறது.
:
யானை பார்க்க அசிங்கமாக இருக்கிறது. நான் பார்க்க அழகாக
இருக்கிறேன். என்னைத் தலைவனாகத் தேர்ந்தெடுங்கள்” என்று
சொன்னது. ஆனால் மற்ற மிருகங்களுக்கு அது பிடிக்கவில்லை.
அதனால் மான் இந்த இடத்திலிருந்து வேறு இடத்துக்குப் போனது. ஒரு

button clicked
lang_hint: ['ta']


/var/folders/4n/6vck65rd73d5bflh0f914j3m0000gn/T/ipykernel_73995/105359903.py:13: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  image = types.Image(content=cv2.imencode('.jpg', path)[1].tostring())


ஓநாய்
வசமாக
சிக்கிக்கொள்
கழுதை
6
கடி
விநாடி / வினாடி



SystemExit: 0

/Users/poorv/.pyenv/versions/3.9.7/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
